## POSTGRESQL 설치 방법
처음 postgresql을 사용하는 경우, Psycopg2와 postgres를 설치하여 super user 선언 및 사용자 권한 부여를 진행해야함. 다만, 주피터 노트북에서 사용하는 경우 설치 및 실행 과정이 다름.

- **Psycopg2-binary 설치**(PostgreSQL과 연결하는 Python Package)
> ```bash
> pip install psycopg2-binary
> conda install -c conda-forge postgresql
> psql --version
> ```

- **PSQL 설치** (PostgreSQL 그 자체)
> ```bash
> conda install -c conda-forge postgresql
>```
- 설치 완료 후 버전 확인
>``` bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> conda list | findstr postgresql
> postgresql                16.3                 h7f155c9_0    conda-forge
> ```
 
- **pgdata** 받아오기
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\initdb" -D "./pgdata"
> ...(log 중략)...
> Success. You can now start the database server using:
>
>   "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l logfile start
> ```
실행 이후 해당 프로젝트 내부에 *pgdata* 디렉토리가 만들어졌는지 확인할 것.

## POSTGRESQL 사용자 선언
SQL을 사용하기 위헤서는 Super User와 하부 사용자가 필요. 여기서 Super User의 이름은 *postgres*로, 하부 사용자 이름은 *d519*로 설정. 먼저, 암호 코드 설정
**1. 암호 파일 생성**
> ```bash
> echo "d519" > superuser_password.txt
> ```

**2. 오류 방지를 위해 기존의 **pgdata** 삭제 (삭제 안될 경우 파일 탐색기에서 pgdata파일 삭제)**
>```bash
> rmdir /s pgdata
> ```

**3. 새로 초기화**
> ```bash
> initdb -D ./pgdata --auth-local=md5 --auth-host=md5 --username=postgres --pwprompt
> ```
- 실행 시 로그
> > ```bash
> > (branchPJH) PS C:\junha\Tilde_Chatbot> initdb -D ./pgdata --auth-local=md5 --auth-host=md5 --username=postgres --pwprompt
> > The files belonging to this database system will be owned by user "overr".
> > This user must also own the server process.
> > 
> > The database cluster will be initialized with locale "Korean_Korea.949".
> > Encoding "UHC" implied by locale is not allowed as a server-side encoding.
> > The default database encoding will be set to "UTF8" instead.
> > initdb: could not find suitable text search configuration for locale "Korean_Korea.949"
> > The default text search configuration will be set to "simple".
> > 
> > Data page checksums are disabled.
> > 
> > Enter new superuser password: 
> > Enter it again:
> > 
> > creating directory pgdata ... ok
> > creating subdirectories ... ok
> > selecting dynamic shared memory implementation ... windows
> > selecting default max_connections ... 100
> > selecting default shared_buffers ... 128MB
> > selecting default time zone ... Asia/Seoul
> > creating configuration files ... ok
> > running bootstrap script ... ok
> > performing post-bootstrap initialization ... ok
> > syncing data to disk ... ok
> > 
> > Success. You can now start the database server using:
> > 
> >     ^"C^:^\Users^\overr^\anaconda3^\envs^\branchPJH^\Library^\bin^\pg^_ctl^" -D ./pgdata -l logfile start
> > ```
   
**4. `initdb` 명령 실행**
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start
> ```
- 실행 시 로그
> > ```bash
> > waiting for server to start.... done
> > server started
> > ```

# PostgreSQL 내부 작업

**1. Postgres 실행 확인**
> ```bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> netstat -an | Select-String "5432"
> TCP    127.0.0.1:5432         0.0.0.0:0              LISTENING
> TCP    [::1]:5432             [::]:0                 LISTENING
> ```

**2. 로그인**
> ```bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> psql -U postgres -h localhost -p 5432
> Password for user postgres: 
> psql (16.3)
> Type "help" for help.
> postgres=#
> ```
`postgres=#`가 나오면 로그인 성공.

**3. User 추가**
> ```bash
> postgres=# CREATE ROLE d519 WITH LOGIN PASSWORD 'd519';
> CREATE ROLE
> ```

**4. DB 선언**
> ```bash
> postgres=# CREATE DATABASE TildeChatBotDB; 
> CREATE DATABASE
> postgres=# \l
>                                                                List of databases
>       Name      |  Owner   | Encoding | Locale Provider |     Collate      |      Ctype       | ICU Locale | ICU Rules |   Access privileges
> ----------------+----------+----------+-----------------+------------------+------------------+------------+-----------+-----------------------
>  postgres       | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
>  template0      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  template1      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  tildechatbotdb | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
> (4 rows)
> ```

**5. tildechatbotdb 소유주를 postgres에서 d519로 변경**
> ```bash
> postgres=# ALTER DATABASE tildechatbotdb OWNER TO d519;
> ALTER DATABASE
> postgres=# \l
>                                                                List of databases
>       Name      |  Owner   | Encoding | Locale Provider |     Collate      |      Ctype       | ICU Locale | ICU Rules |   Access privileges
> ----------------+----------+----------+-----------------+------------------+------------------+------------+-----------+-----------------------
>  postgres       | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
>  template0      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  template1      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  tildechatbotdb | d519     | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =Tc/d519             +
>                 |          |          |                 |                  |                  |            |           | d519=CTc/d519
> (4 rows)
> ```

**6. 로그아웃**
> ```bash
> postgres=# \q
> (branchPJH) PS C:\junha\Tilde_Chatbot>
> ```


# 전원 다시 시작시
- Terminal에서 PostgerSQL 서버 시작하기  
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start
> ```
 
- Terminal에서 super user인 **postgres**로 로그인
> ```bash
> psql -U postgres -h localhost -p 5432
> ```

In [1]:
import pandas as pd
import psycopg2

In [2]:
# & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start

In [3]:
# Postgres 연결 확인
DB_CONFIG = {
        'host': 'localhost',
        'database': 'tildechatbotdb',
        'user': 'd519',
        'password': 'd519',
        'port': 5432
    }

try:
    # PostgreSQL에 연결
    conn = psycopg2.connect(**DB_CONFIG)
    print("PostgreSQL 연결 성공!")
    conn.close()
except Exception as e:
    print(f"PostgreSQL 연결 실패: {e}")


PostgreSQL 연결 성공!


## 초기 작업 : 엑셀 내용 삽입
2024년 12월 29일 기준으로 PostgreSQL 내부 tildechatbotdb에는 ChatTable이라는 이름의 테이블이 존재. 엑셀 내용이 전부 삽입되어있는 상태임. 만약, 새로 선언한다면, 주석을 제거하고 실행할 것.

In [11]:
#def create_table_from_excel_psycopg2(excel_file, table_name, db_config):
#    df = pd.read_excel(excel_file)
#
#    conn = psycopg2.connect(**db_config)
#    cursor = conn.cursor()
#
#    columns = ', '.join([f'"{col}" TEXT' for col in df.columns])
#    create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns})'
#    cursor.execute(create_table_query)
#
#    for _, row in df.iterrows():
#        column_names = ', '.join([f'"{col}"' for col in df.columns])
#        placeholders = ', '.join(['%s'] * len(row))
#        insert_query = f'INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})'
#        cursor.execute(insert_query, tuple(row.values))
#
#    conn.commit()
#    cursor.close()
#    conn.close()
#    print(f"Table '{table_name}' created and data inserted successfully.")

In [12]:
#EXCEL_FILE = "C:/junha/Datasets/20250212Chatdata.xlsx"
#TABLE_NAME = "ChatTable20250212"
#DB_CONFIG = {
#        'host': 'localhost',
#        'database': 'tildechatbotdb',
#        'user': 'd519',
#        'password': 'd519',
#        'port': 5432
#    }
#
#create_table_from_excel_psycopg2(EXCEL_FILE, TABLE_NAME, DB_CONFIG)

Table 'ChatTable20250212' created and data inserted successfully.


## 데이터베이스 삽입 상태 확인

In [13]:
def verify_data_insertion(table_name, db_config):
    try:
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)

        result = cursor.fetchall()

        if not result:
            print(f"No data found in table '{table_name}'.")
            return None

        column_names = [desc[0] for desc in cursor.description]

        df = pd.DataFrame(result, columns=column_names)
        print(f"Data in table '{table_name}':")

        cursor.close()
        conn.close()
        return df

    except Exception as e:
        print(f"Error verifying data: {e}")


In [14]:
#verify_data_insertion('ChatTable',DB_CONFIG)
verify_data_insertion('ChatTable20250123',DB_CONFIG)
verify_data_insertion('ChatTable20250212',DB_CONFIG)

Data in table 'ChatTable20250123':
Data in table 'ChatTable20250212':


,Data_no,질문수정,나이,성별,몸무게,노출제품유형,노출제품명,노출부위,노출유형
0,1,"""목,기관지에 좋은 약 , 약품에 대한 빨갛게 되는 증상 약,"", ""안녕하세요 , ...",NaN,NaN,NaN,공업용화학제품,약품,발,피부
1,2,"테라플루나이트를 오래 복용하면 간수치가 많이 오르나요?, ""잘못된걸 알지만수면 목적...",NaN,NaN,NaN,의약품,테라플루나이트,NaN,경구
2,3,"전자담배 목 간지러움, 일회용 전자담배 중에서 비ㅇ치라고 있는데 그 브랜드 복숭아 ...",20살,NaN,NaN,중독성식품,전자담배,목구멍,흡입
3,4,"지네한테 물렸어요, 아주 작은 한 5cm정도되는 지네한테 물렷어요 대가리는 갈색이고...",NaN,NaN,NaN,독성있는동식물,지네,NaN,피부
4,5,"아스팔트 섭취, ""아침에 빵을 먹다가 아스팔트 도로위에 떨었뜨렸는데 떨어진 부분만 ...",NaN,NaN,NaN,이물질섭취,아스팔트,NaN,경구
...,...,...,...,...,...,...,...,...,...
2007,2013,"저는 28살 여자이고요, 평소 두통 때문에 이부프로펜 400mg으로 된 진통제를 복...",28살,여자,NaN,의약품,이부프로펜,NaN,경구
2008,2014,저는 38살 여자입니다. 요즘 피곤하고 잠을 잘 못 자는 날이 많아서 친구가 준 졸...,38살,여자,NaN,의약품,졸피뎀,NaN,경구
2009,2015,저는 24살 여자이고 요즘 불안이 너무 심해서 벤조디아제핀 계열의 약물을 복용 중이...,24살,여자,NaN,의약품,벤조디아제핀,NaN,경구
2010,2016,저는 27살 여자입니다. 최근 다이어트를 위해 주변 추천으로 한 다이어트 보조제를 ...,27살,여자,NaN,의약품,로카세린,NaN,경구


## 원하는 값만 추출

In [15]:
def select_distinct_values(query, db_config):
    try:
        connection = psycopg2.connect(**db_config)
        cursor = connection.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        
        distinct_values = [row[0] for row in results]
        return distinct_values
    except Exception as e:
        print("Database error:", e)
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [17]:
TBL_NAME = "ChatTable20250212"
COLUMN_NAME = "노출유형"
QUERY = f"SELECT DISTINCT {COLUMN_NAME} FROM {TBL_NAME};"
distinct_values = select_distinct_values(QUERY, DB_CONFIG)
distinct_values

['흡입, 피부',
 '경구',
 '피부, 흡입',
 '안구, 피부, 흡입',
 '피부, 안구, 경구',
 '안구, 피부',
 '피부, 기타',
 '흡입, 경구',
 '피부, 흡입, 경구',
 '경구, 기타',
 '피부, 안구, 흡입',
 '피부, 안구',
 '피부, 경구',
 '기타',
 '안구, 흡입, 경구',
 '피부, 경구, 흡입',
 '안구',
 '경구, 피부',
 '피부',
 '흡입',
 '안구, 경구, 흡입',
 '안구, 흡입',
 '안구, 경구',
 '안구, 흡입, 피부',
 '경구, 흡입',
 '흡입, 안구, 피부',
 '흡입, 안구']

In [12]:
def search_keyword(question):
    matching_words = [value for value in distinct_values if value in question]
    if matching_words:
        return matching_words
    else:
        return None

In [13]:
def return_searched_keyword(table_name, db_config, column_name, keywords):
    try:
        if not keywords:
            print("No keywords provided.")
            return None

        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        like_conditions = " OR ".join([f"{column_name} LIKE '%{keyword}%'" for keyword in keywords])
        query = f"SELECT * FROM {table_name} WHERE {like_conditions};"

        cursor.execute(query)
        result = cursor.fetchall()

        if not result:
            print(f"No data found in table '{table_name}' for keywords: {keywords}.")
            return None

        column_names = [desc[0] for desc in cursor.description]

        df = pd.DataFrame(result, columns=column_names)
        cursor.close()
        conn.close()

        return df

    except Exception as e:
        print(f"Error verifying data: {e}")


In [15]:
TBL_NAME = "ChatTable"
COLUMN_NAME = "노출제품명"
QUERY = f"SELECT DISTINCT {COLUMN_NAME} FROM {TBL_NAME};"
question = "우리 아이가 농약(그라목손)을 먹었어요. 어떡해요!"
KEYWORD = search_keyword(question)

return_searched_keyword(TBL_NAME, DB_CONFIG, COLUMN_NAME, KEYWORD)

,Data_no,구분,데이터제공,임상과,제목,질문원본,질문수정,나이,성별,노출제품유형,...,노출시간,노출후증상,몸무게,노출용량,노출후응급처치,답변원본,답변수정,답변생성1,Date,Doctor
0,512,일반인,생성,NaN,NaN,NaN,"저희 할아버지(75세, 남성)가 오늘 아침에 농장에서 사용 중이던 공업용 농약이 실...",75세,남성,농약,...,1분,"뺨 피부가 빨갛게 부어오르며 따갑고 화끈거림, 메스꺼움, 약간의 어지러움",NaN,NaN,물로 세척하고 닦아냄,NaN,NaN,"농약은 피부 접촉, 흡입, 섭취 등 다양한 경로로 인체에 유해할 수 있는 화학물질로...",NaN,gpt-4o-2024-11-20
1,304,일반인,생성,NaN,NaN,NaN,"저희 할아버지(76세, 남성) 얘긴데요. 며칠 전에 농약이 들어 있는 병을 실수로 ...",76세,남성,농약,...,20분,"손이 빨갛게 부어오르고 따가움, 손바닥에 물집 발생",NaN,NaN,물로 바로 씻어냄,NaN,NaN,"농약이 피부에 접촉된 후 손이 빨갛게 부어오르고 물집이 생겼다면, 이는 농약의 화학...",NaN,gpt-4o-2024-11-20
2,312,일반인,생성,NaN,NaN,NaN,"제가 조금 걱정되는 일이 있어서 문의드립니다. 저희 아버지가 72세이시고, 어제 오...",72세,남자,농약,...,일회성,"피부가 빨갛게 부어오르고 따가운 느낌, 어깨 근육통",NaN,NaN,"약 5분 정도 세척, 소독용 연고 사용",NaN,NaN,"농약에 노출된 후 피부에 발적, 부종, 따가움 등의 증상이 나타난 것은 농약 성분에...",NaN,gpt-4o-2024-11-20
3,320,일반인,생성,NaN,NaN,NaN,70대 남성입니다. 어제 오후 정원을 세우려고 분무기에 농약을 넣고 화초에 뿌리다가...,70대,남성,농약,...,30분,"목 따끔거림, 약간의 기침, 눈 욱신거림, 피부 간지러움",NaN,조금,NaN,NaN,NaN,농약에 노출된 상황에서는 신속한 응급처치와 증상 관찰이 중요합니다. 농약의 성분을 ...,NaN,gpt-4o-2024-11-20
4,359,일반인,생성,NaN,NaN,NaN,"저희 할아버지께서 72세인데, 농약으로 인한 중독 증상이 아닌가 걱정돼서 질문드려요...",72세,남자,농약,...,일회성,속이 메스껍고 약간의 어지럼증,NaN,적은 양,NaN,NaN,NaN,농약에 노출되었을 가능성이 있는 상황에서 할아버지께서 메스꺼움과 어지럼증을 호소하신...,NaN,gpt-4o-2024-11-20
5,363,일반인,생성,NaN,NaN,NaN,72세 남자입니다. 제가 어제 오후쯤 농약을 묻은 장갑으로 실수로 입을 닦았습니다....,72세,남자,농약,...,일회성,"입술과 혀에 찌릿찌릿한 느낌, 위가 살살 아프고 메스꺼움",NaN,NaN,물로 입을 헹굴음,NaN,NaN,"농약은 일반적으로 살충제, 제초제, 살균제 등 다양한 화학물질로 구성되어 있으며, ...",NaN,gpt-4o-2024-11-20
6,378,일반인,생성,NaN,NaN,NaN,72세 남성입니다. 오늘 아침에 제가 잡초 제거하려고 농약을 뿌리면서 20분 정도 ...,72세,남자,농약,...,20분,"콧속 화끈거림, 머리 어지러움, 무거운 느낌, 두통",NaN,NaN,작업 후 바로 물로 얼굴을 씻음,NaN,NaN,농약에 노출된 상황에서 나타나는 증상은 농약의 성분과 노출된 양에 따라 다를 수 있...,NaN,gpt-4o-2024-11-20
7,390,일반인,생성,NaN,NaN,NaN,37세 여성인데 어제 오후에 집 근처 텃밭에서 농약을 살포하다가 바람이 불어서 얼굴...,37세,여성,농약,...,15분,"목과 팔이 따갑고 붉게 부어오름, 현기증, 머리가 무거운 느낌",NaN,NaN,바로 물로 씻어냄,NaN,NaN,"농약에 노출된 후 피부 자극과 전신 증상이 나타난 경우, 이는 농약의 성분에 따라 ...",NaN,gpt-4o-2024-11-20
8,392,일반인,생성,NaN,NaN,NaN,저희 아이는 3살이고 몸무게는 12kg입니다. 오늘 오후에 밖에서 놀다가 이웃집에서...,3세,NaN,농약,...,10분,"기침, 눈가가 빨개짐, 콧물, 눈을 비빔",12kg,NaN,"얼굴을 닦아줌, 물을 먹임",NaN,NaN,"농약은 종류에 따라 독성이 다르며, 흡입, 피부 접촉, 또는 섭취를 통해 인체에 영...",NaN,gpt-4o-2024-11-20
9,401,일반인,생성,NaN,NaN,NaN,20대 초반 여자입니다. 어제 농촌에서 자원봉사를 하다가 농작물을 보호하기 위해 뿌...,20대 초반,여자,농약,...,30분,"팔뚝 부분이 붉어지고 따끔거림, 머리 어지러움, 경미한 두통",NaN,NaN,작업 후 바로 물로 씻어냄,NaN,NaN,"농약 작업 중 피부와 접촉하거나 흡입한 경우, 농약의 종류와 성분에 따라 증상이 다...",NaN,gpt-4o-2024-11-20
